In [7]:
from pyscf import gto, scf, mcscf, fci
import numpy as np
import matplotlib.pyplot as plt
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import os 

Use this to delete the temp files BEFORE a dmrg calculation

In [18]:
folder_path = "./tmpheh"  #change here the directory where the tmp files are located

for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory) and delete it
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")

Deleted: ./tmpheh/F.MPS.GS.0
Deleted: ./tmpheh/F.MPS.GS.1
Deleted: ./tmpheh/F.MPS.INFO.GS.LEFT.0
Deleted: ./tmpheh/F.MPS.INFO.GS.RIGHT.0
Deleted: ./tmpheh/F.MPS.INFO.GS.LEFT.1
Deleted: ./tmpheh/F.MPS.INFO.GS.RIGHT.1
Deleted: ./tmpheh/F.MPS.INFO.GS.LEFT.2
Deleted: ./tmpheh/F.MPS.INFO.GS.RIGHT.2
Deleted: ./tmpheh/F.MPS.GS.-1
Deleted: ./tmpheh/GS-mps_info.bin


In [17]:
#Helium-hydrogen test test

heh = gto.M(atom=[["He", 0, 0, 0],
                  ["H", 1.2, 0, 0]], basis = 'sto-3g', charge = 0, spin = 1)
uhf_heh=scf.UHF(heh)
uhf_heh.kernel()

Heh_alpha_mo_en = uhf_heh.mo_energy[0]
Heh_beta_mo_en = uhf_heh.mo_energy[1]
Heh_alpha_mo_coeff = uhf_heh.mo_coeff[0]
Heh_beta_mo_coeff = uhf_heh.mo_coeff[1]
Heh_alpha_occ=uhf_heh.mo_occ[0]
Heh_beta_occ=uhf_heh.mo_occ[1]

#print(Heh_alpha_mo_en)
#print(Heh_beta_mo_en)
#print(Heh_alpha_occ)
#print(Heh_beta_occ)
print(Heh_alpha_mo_coeff)
print(Heh_beta_mo_coeff)


#np.save('/workspaces/MRA-OrbitalOptimization/Block2/HeH_test/coeffs/120_alphacoeffs.npy', Heh_alpha_mo_coeff)
#np.save('/workspaces/MRA-OrbitalOptimization/Block2/HeH_test/coeffs/120_betacoeffs.npy', Heh_beta_mo_coeff)

converged SCF energy = -3.21609329134427  <S^2> = 0.75  2S+1 = 2
[[ 0.92952054 -0.47557501]
 [ 0.18813994  1.02702648]]
[[ 0.9751129  -0.37327586]
 [ 0.07705034  1.04127003]]


In [12]:
#FCI Referenz
Heh_fci = gto.M(atom=[["He", 0, 0, 0],
                    ["H", 1.2, 0, 0]], basis = 'cc-pvtz', charge = 0, spin = 1)
uhf_hehfci=scf.UHF(Heh_fci)
uhf_hehfci.kernel()

fcisolver = fci.FCI(uhf_hehfci)
fcisolver.kernel()

converged SCF energy = -3.31347707234695  <S^2> = 0.75072075  2S+1 = 2.0007206


(-3.3570239958160712,
 FCIvector([[ 9.95118461e-01,  7.92832067e-03,  5.08386742e-03, ...,
              1.01541275e-13, -7.78722173e-14, -1.28399633e-04],
            [ 8.50002823e-04,  1.11067411e-02,  1.87188877e-02, ...,
             -2.64358885e-14,  2.03964624e-14,  3.03392541e-05],
            [ 1.83140000e-04, -3.54199245e-03, -8.19702535e-03, ...,
              1.20824447e-14, -9.30201877e-15,  8.28426400e-04],
            ...,
            [-4.55999251e-04, -4.98784561e-05,  4.01668097e-07, ...,
             -3.68796020e-14,  2.82986633e-14, -6.23345353e-06],
            [ 7.96757212e-17, -2.18760647e-17, -3.48338605e-18, ...,
             -6.92420072e-06, -9.02049408e-06,  3.49357743e-20],
            [ 1.31636205e-12, -7.72982854e-13, -6.13096372e-13, ...,
             -9.02049408e-06,  6.92420072e-06, -2.40562062e-11]]))

In [15]:
heh = gto.M(atom=[["He", 0, 0, 0],
                  ["H", 1.2, 0, 0]], basis = 'sto-3g', charge = 0, spin = 1)
uhf_heh=scf.UHF(heh)
uhf_heh.kernel()

norbs, nelecas = (2, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf_heh, norbs, nelecas).mc1step()[0]

bond_dims = [250] * 8
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf_heh,
    ncore=0, ncas=None, g2e_symm=1)


driver_631g = DMRGDriver(scratch="./tmpheh", symm_type=SymmetryTypes.SZ, n_threads=4)
driver_631g.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo_631g = driver_631g.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket_631g = driver_631g.get_random_mps(tag="GS", bond_dim=250, nroots=1)
energy_631g = driver_631g.dmrg(mpo_631g, ket_631g, n_sweeps=10, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energy_631g)
print('difference dmrg UCASSCF', ref_ucas - energy_631g)

converged SCF energy = -3.21609329134427  <S^2> = 0.75  2S+1 = 2
UCASSCF energy = -3.21609329134427
UCASCI E = -3.21609329134427  E(CI) = -4.09805530954427  S^2 = 0.7500000
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =         54

Build MPO | Nsites =     2 | Nterms =         54 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     2 .. Mmpo =    22 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     2 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.003 Tmvc-total = 0.000 MPO bond dimension =    22 MaxDW = 0.00e+00
NNZ =           44 SIZE =           44 SPT = 0.0000



Rank =     0 Ttotal =      0.007 MPO method = FastBipartite bond dimension =      22 NNZ =           44 SIZE =           44 SPT = 0.0000

Sweep =    0 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.025 | E =      -3.2160932913 | DW = 0.00000e+00

Sweep =    1 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.048 | E =      -3.2160932913 | DE = 0.00e+00 | DW = 0.00000e+00

Sweep =    2 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.073 | E =      -3.2160932913 | DE = 0.00e+00 | DW = 0.00000e+00

Sweep =    3 | Direction = backward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.099 | E =      -3.2160932913 | DE = 0.00e+00 | DW = 0.00000e+00

Sweep =    4 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-05 | Dav threshold = 

In [ ]:
heh = gto.M(atom=[["He", 0, 0, 0],
                  ["H", 1.2, 0, 0]], basis = '6-31g', charge = 0, spin = 1)
uhf_heh=scf.UHF(heh)
uhf_heh.kernel()

Heh_alpha_mo_coeff = uhf_heh.mo_coeff[0]
Heh_beta_mo_coeff = uhf_heh.mo_coeff[1]

print(Heh_alpha_mo_coeff)
print(Heh_beta_mo_coeff)



converged SCF energy = -3.30266757793893  <S^2> = 0.75050081  2S+1 = 2.0005007
[[ 0.58020031 -0.20591684  0.05351912 -1.16642259]
 [ 0.46226609 -0.38568545 -0.34841238  1.3748839 ]
 [ 0.10484983  0.43270529 -1.25642272 -0.11647102]
 [ 0.06458282  0.76354335  1.30941201 -0.29174708]]
[[ 0.57996007 -0.14740682 -0.07181091 -1.17439382]
 [ 0.50430106 -0.4081216   0.28469455  1.36828591]
 [ 0.04549436  0.19389067  1.31798407 -0.11681424]
 [ 0.0218849   1.00009922 -1.14045569 -0.29245336]]
